In [1]:
import torch
import torchvision as vision
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
#torch.cuda.set_device(0)

In [2]:
# change the PIL to tensor
from torchvision.transforms import ToTensor, Normalize as Norm, Compose
transform = Compose([ToTensor(),Norm((0.5,), (0.5,))]) # mean, range (change to 0 - 1)

train_dataset = MNIST(root='../dataset/MNIST',train=True,download=True,transform=transform)
test_dataset = MNIST(root='../dataset/MNIST',train=False,download=True,transform=transform)
train_feeder = DataLoader(train_dataset, batch_size=64,shuffle=True, num_workers=2)
test_feeder = DataLoader(test_dataset,batch_size=64,shuffle=False,num_workers=2)

In [3]:
from torch.nn import Conv2d as Conv
from torch.nn import MaxPool2d as Pool
from torch.nn import Linear as FC
from torch.nn.functional import relu, softmax
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = Conv(1, 6, kernel_size=5)
        self.pool = Pool(2, 2)
        self.conv2 = Conv(6, 16, kernel_size=5)
        self.fc1 = FC(16 * 4 * 4, 120)
        self.fc2 = FC(120, 84)
        self.fc3 = FC(84, 10)

    def forward(self, x):
        x = self.pool(relu(self.conv1(x)))
        x = self.pool(relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4) # in pytorch: need to view to reshape
        x = relu(self.fc1(x))
        x = relu(self.fc2(x))
        x = self.fc3(x)
        x = softmax(x)
        return x
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
net = Net().to(device)

In [4]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

criterion = CrossEntropyLoss()
optimizer = SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
for epoch in range(20):
    running_loss = 0.0
    for _, (inputs, labels) in enumerate(train_feeder,0): # python enumerate: turn a iterable obj to a list
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() # refresh the buffer
        outputs = net(inputs) # forward
        loss = criterion(outputs, labels) # calculate the loss
        loss.backward() # BP
        optimizer.step() # p' = p - lr * grad
        running_loss += loss.item()
    #if epoch % 1000 == 0:
    print('epoch {} loss: {:.3f}'.format(epoch + 1, running_loss))

/home/zeroweight/.local/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 1 loss: 2159.630
epoch 2 loss: 2159.189
epoch 3 loss: 2158.533


Process Process-8:
Process Process-7:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/zeroweight/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zeroweight/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/zeroweight/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 57, in _wo

RuntimeError: DataLoader worker (pid 19995) exited unexpectedly with exit code 1.